### Getting data from the Google Earth Engine

In [1]:
# importing packages and initializing Google Earth Engine
%matplotlib inline
import matplotlib.pyplot as plt
import ee
import urllib
import zipfile
import os

ee.Initialize()

In [2]:
state_name = 'Oregon'
year = '2010'
folder_loc = '/Users/patrickdoupe/Dropbox/insightDataScience/project/'

if not os.path.exists(folder_loc+'LANDSAT_TOA/'+state_name):
    os.makedirs(folder_loc+'LANDSAT_TOA/'+state_name)

In [3]:
## Getting Satellite data for Oregon
# Load LANDSAT TOA image for 2010
image = ee.Image('LANDSAT/LE7_TOA_1YEAR/'+year) 

# Get State boundaries
featureCollection = ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8');
# Get a map of the state
state_filter = featureCollection.filter(ee.Filter.eq('Name', state_name));
border = ee.Feature(state_filter.first())

# Make a bitmask that we can use to set the values outside the
# region to zero (for download), and then also call clip() to 
# set the mask (for things like addToMap() that use the mask)
outside = ee.Image(1).uint8().paint(border, 0);
image = image.where(outside, 0).clip(border);

In [12]:
# Exporting path
path = image.getDownloadURL({
    'scale': 100,
    'crs': 'EPSG:4326'
    });

spatial_data_file = '2010.zip'
a = urllib.urlretrieve(path, spatial_data_file)
print a
zfile = zipfile.ZipFile(spatial_data_file)
zfile.extractall('LANDSAT_TOA/'+state_name)

('2010.zip', <httplib.HTTPMessage instance at 0x10508f128>)


In [ ]:
# Cleaning up the names of the files
channels = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6_VCID_2', 'B7']

for filename in os.listdir('LANDSAT_TOA/'+state_name):
    print filename
    for channel in channels:
        try:
            prefix, band, extension = filename.partition(channel)
            os.rename(
                'LANDSAT_TOA/'+state_name+'/'+filename,
                'LANDSAT_TOA/'+state_name+'/'+state_name+'_'+year+'_'+band+extension)
            count += 1
        except:
            pass

In [ ]:
for filename in os.listdir(folder_loc+'LANDSAT_TOA/'+state_name):
    print filename